# Testing polycentric gyration
This script tests the code for finding poly centric gyration from the paper [From centre to centres: polycentric structures in individual mobility](https://arxiv.org/abs/2108.08113). Code is from github https://github.com/rohit-sahasrabuddhe/polycentric-mobility . Functions are taken from the main.py file, It would be better to source them in or run the script directly but I don't know how.

# Setup
The packages and functions used by the script

In [3]:
import numpy as np
import pandas as pd
import geopandas as gp
from sklearn.cluster import KMeans
from haversine import haversine_vector
from sklearn.metrics import auc
from scipy.spatial import distance_matrix as DM

from joblib import Parallel, delayed

# Functions for conversion from latlon to cartesian and back
def to_cartesian(lat, lon):
    lat, lon = np.pi * lat / 180, np.pi * lon / 180
    return np.cos(lat) * np.cos(lon), np.cos(lat) * np.sin(lon), np.sin(lat)
def to_latlon(x,y,z):
    lat, lon = np.arctan2(z, np.sqrt(x**2+y**2))*180/np.pi, np.arctan2(y, x)*180/np.pi
    return lat, lon

class TrimmedKMeans:
    def __init__(self, k, data, weights, cutoff):
        self.k = k
        self.data = data #A numpy array of size [N, 3]
        self.weights = weights / np.sum(weights) #size [N,]
        self.centers = self.data[np.random.choice(range(self.data.shape[0]), size=k, replace=False)]
        
        self.distance_matrix = DM(self.data, self.centers)
        self.cluster_assignment = np.argmin(self.distance_matrix, axis=1)
        self.distance = np.min(self.distance_matrix, axis=1)
        self.inertia = 0
        
        self.cutoff=cutoff
        
    def get_inertia_labels(self):
        self.distance_matrix = DM(self.data, self.centers)
        self.cluster_assignment = np.argmin(self.distance_matrix, axis=1)
        self.distance = np.min(self.distance_matrix, axis=1)
        self.inertia = 0
        for i in range(self.k): # Loop through all the clusters
            # get the coordinates, global weights and distance to center
            coords, weights, dists = self.data[self.cluster_assignment == i], self.weights[self.cluster_assignment == i], self.distance[self.cluster_assignment == i]
            if coords.shape[0] == 0:
                continue
            
            indices_asc = np.argsort(dists)
            coords, weights, dists = coords[indices_asc], weights[indices_asc], dists[indices_asc] # sort everything by the distance
            cluster_wt = np.sum(weights) # total weight of the cluster
            weights = weights / cluster_wt # this gives the local weight (within the cluster)
            weights_cumsum = np.cumsum(weights)
            
            last_entry = np.sum(weights_cumsum <= self.cutoff) + 1 # the index of the last location that needs to be looked at
            coords, weights, dists, weights_cumsum = coords[:last_entry].copy(), weights[:last_entry].copy(), dists[:last_entry].copy(), weights_cumsum[:last_entry].copy()
            # Remove the extra weight
            weights[-1] -= weights_cumsum[-1] - self.cutoff
            # Add to the inertia
            self.inertia += np.sum((weights * cluster_wt) * (dists**2))
        return np.sqrt(self.inertia), self.cluster_assignment
        
    def update(self):
        self.distance_matrix = DM(self.data, self.centers)
        self.cluster_assignment = np.argmin(self.distance_matrix, axis=1)
        self.distance = np.min(self.distance_matrix, axis=1)
        
        for i in range(self.k): # Loop through all the clusters
            # get the coordinates, global weights and distance to center
            coords, weights, dists = self.data[self.cluster_assignment == i], self.weights[self.cluster_assignment == i], self.distance[self.cluster_assignment == i]
            if coords.shape[0] == 0:
                continue
            
            indices_asc = np.argsort(dists)
            coords, weights, dists = coords[indices_asc], weights[indices_asc], dists[indices_asc] # sort everything by the distance
            cluster_wt = np.sum(weights) # total weight of the cluster
            weights = weights / cluster_wt # this gives the local weight (within the cluster)
            weights_cumsum = np.cumsum(weights)
            # last entry is the index of the last location that needs to be looked at
            last_entry = np.sum(weights_cumsum <= self.cutoff) + 1
            coords, weights, dists, weights_cumsum = coords[:last_entry].copy(), weights[:last_entry].copy(), dists[:last_entry].copy(), weights_cumsum[:last_entry].copy()
            # Remove the extra weight
            weights[-1] -= weights_cumsum[-1] - self.cutoff
            
            # Update the center
            weights = weights / np.sum(weights)
            self.centers[i] = np.average(coords, axis=0, weights=weights)       
        

    def plot(self):
        for i in range(self.k):
            plt.scatter(self.data[self.cluster_assignment == i][:, 0], self.data[self.cluster_assignment == i][:, 1])
        plt.scatter(self.centers[:, 0], self.centers[:, 1], marker='+', color='black', s=50)
    
    def get_best_fit(self):
        best_centers, best_inertia, best_labels = None , np.inf, None
        for _ in range(50): #compare across 50 random initializations
            c = np.inf
            self.centers = self.data[np.random.choice(range(self.data.shape[0]), size=self.k, replace=False)]
            for _ in range(50): #fixed number of iterations
                old_c = np.copy(self.centers)
                self.update()
                c = np.sum((self.centers - old_c)**2)
                if c == 0:
                    break
            this_inertia, this_labels = self.get_inertia_labels()
            if this_inertia < best_inertia:
                best_inertia = this_inertia
                best_labels = this_labels
                best_centers = self.centers
            if best_inertia == 0:
                break
            
        return best_centers, best_labels, best_inertia
    

def get_result(u, user_data, locs, max_k, trimming_coeff):
    #print(f"User {u}, {to_print}")
    result = {'user':u, 'com':None, 'tcom':None, 'rog':None, 'L1':None, 'L2':None, 'k':None, 'centers':None, 'auc_com':None, 'auc_1':None, 'auc_2':None, 'auc_k':None, 'auc_kmeans':None}
    def get_area_auc(x, k, max_area, df):
        centers = x
        dists = np.min(haversine_vector(list(df.coords), centers, comb=True), axis=0)
        df['distance'] = dists
        df['area'] = k * df['distance']**2
        df = df.sort_values('area')[['area', 'time_spent']]        
        df = df[df['area'] <= max_area]
        if df.empty:
            return 0        
        df.time_spent = df.time_spent.cumsum()        
        df['area'] = df['area'] / max_area
        x = [0] + list(df['area']) + [1]
        y = [0] + list(df.time_spent) + [list(df.time_spent)[-1]]
        return auc(x, y)
        
    user_data = user_data[['loc', 'time_spent']].groupby('loc').sum()
    try:
        user_data.time_spent = user_data.time_spent.dt.total_seconds()
    except:
        pass
    user_data.time_spent = user_data.time_spent / user_data.time_spent.sum()
    user_data['lat'] = locs.loc[user_data.index].lat
    user_data['lon'] = locs.loc[user_data.index].lon
    
    highest_gap = None
    best_auc = None
    best_gap = None
    best_k = 1
    best_centers = None
    
    
    user_data['coords'] = list(zip(user_data.lat, user_data.lon))        
    user_data['x'], user_data['y'], user_data['z'] = to_cartesian(user_data['lat'], user_data['lon'])
    com = to_latlon(np.sum(user_data['x']*user_data.time_spent), np.sum(user_data['y']*user_data.time_spent), np.sum(user_data['z']*user_data.time_spent))
    dist = haversine_vector(list(user_data.coords), [com], comb=True)
    rog = np.sqrt(np.sum(user_data.time_spent.to_numpy() * (dist**2)))
    com_auc = get_area_auc(com, 1, rog**2, user_data.copy())
    
    
    result['com'] = com
    result['rog'] = rog
    result['L1'], result['L2'] = list(user_data.sort_values('time_spent', ascending=False).coords[:2])
    result['auc_com'] = com_auc
    
    
    train_data_list = []
    # find max min and shape outside loop
    lat_min, lat_max = user_data.lat.min(), user_data.lat.max()
    lon_min, lon_max = user_data.lon.min(), user_data.lon.max()
    size = user_data.shape[0]
    for i in range(50):
        train_data = user_data.copy()
        train_data['lat'] = np.random.uniform(low=lat_min, high=lat_max, size=size)
        train_data['lon'] = np.random.uniform(low=lon_min, high=lon_max, size=size)
        train_data['coords'] = list(zip(train_data.lat, train_data.lon))        
        train_data['x'], train_data['y'], train_data['z'] = to_cartesian(train_data['lat'], train_data['lon'])
            
        #find rog of this data
        com = to_latlon(np.sum(train_data['x']*train_data.time_spent), np.sum(train_data['y']*train_data.time_spent), np.sum(train_data['z']*train_data.time_spent))
        dist = haversine_vector(list(train_data.coords), [com], comb=True)
        train_rog = np.sqrt(np.sum(train_data.time_spent.to_numpy() * (dist**2)))   
        
        train_data_list.append((train_data, train_rog))
    
    
    for k in range(1, max_k+1):   
        Trim = TrimmedKMeans(k, user_data[['x','y', 'z']].to_numpy(), weights = user_data.time_spent.to_numpy(), cutoff=trimming_coeff)
        true_centers, _, _ = Trim.get_best_fit()        
        true_centers = np.array([np.array(to_latlon(*i)) for i in true_centers])
        true_auc = get_area_auc(true_centers, k, rog**2, user_data.copy())
        
        if k == 1:
            result['tcom'] = tuple(true_centers[0])
            result['auc_1'] = true_auc
        if k== 2:
            result['auc_2'] = true_auc
        
        new_aucs = []
        for train_data, train_rog in train_data_list:
            Trim = TrimmedKMeans(k, train_data[['x','y', 'z']].to_numpy(), weights = train_data.time_spent.to_numpy(), cutoff=trimming_coeff)
            centers, _, _ = Trim.get_best_fit()        
            centers = np.array([np.array(to_latlon(*i)) for i in centers])
            new_aucs.append(get_area_auc(centers, k, train_rog**2, train_data.copy()))
            
            
        new_mean = np.mean(new_aucs)
        new_std = np.std(new_aucs)        
        gap = true_auc - new_mean
        
        if k == 1:
            highest_gap = gap
            best_gap = gap
            best_auc = true_auc
            best_centers = true_centers
            best_k = 1
            continue
        
        
        if gap - new_std > highest_gap:
            best_auc = true_auc
            best_gap = gap
            best_centers = true_centers
            best_k = k
        highest_gap = max(highest_gap, gap)
    
    result['k'] = best_k
    result['auc_k'], result['centers'] = best_auc, list(best_centers)
    
    kmeans = KMeans(result['k'])
    kmeans.fit(user_data[['x','y', 'z']].to_numpy(), sample_weight = user_data.time_spent.to_numpy())
    kmeans_centers = np.array([np.array(to_latlon(*i)) for i in kmeans.cluster_centers_])
    result['auc_kmeans'] = get_area_auc(kmeans_centers, result['k'], rog**2, user_data.copy())
    return result

def main(data_path, results_path="demo_results.pkl", max_k=6, trimming_coeff=0.9):
    data = pd.read_pickle(data_path)
    try:
        data['time_spent'] = data['end_time'] - data['start_time']
    except:
        pass
    user_list = sorted(data.user.unique())
    locs = data[['loc', 'lat', 'lon']].groupby('loc').mean().copy()
    
    result = pd.DataFrame(Parallel(n_jobs=-1)(delayed(get_result)(u, data[data.user == u], locs, max_k, trimming_coeff) for u in user_list)).set_index('user')
    result.to_pickle(results_path)
    return result

# load mobility csv and save as pandas dataframe

These code chunks create the appropriately formatted dataframe from a csv of the mobility data produced in R

In [2]:
dis_pandas = pd.read_csv("/home/jonno/COVID_project/COVID_project_data/poly_df.csv").loc[:,['loc','lat','lon', 'time_spent', 'user']]
dis_pandas['time_spent'] = dis_pandas['time_spent'].astype('float')
dis_pandas.to_pickle("/home/jonno/COVID_project/COVID_project_data/poly_df.pkl")
del dis_pandas

          loc        lat       lon  time_spent  user
0         842  51.521712 -0.047056           1   172
1         842  51.521971 -0.046572           1   390
2         847  51.521690 -0.047084           1   263
3          74  51.469348  0.119686           1    69
4         841  51.521621 -0.043450           1   710
...       ...        ...       ...         ...   ...
24936986  242  51.528828 -0.349071           1   242
24936987  376  51.478107 -0.194349           1   376
24936988  128  51.412014 -0.034207           1   133
24936989   46  51.601814 -0.235075           1    34
24936990  295  51.620292 -0.070886           1   282

[24936991 rows x 5 columns]


In [13]:
#create smaller file that will be easier to test
user_list = [i for i in range(10)]
dis_pandas[dis_pandas["user"].isin(user_list)].to_pickle("/home/jonno/COVID_project/COVID_project_data/poly_df2.pkl")

## Set file paths

In [4]:
script_path = "/home/jonno/polycentric-mobility/main.py"
target_file_path = "/home/jonno/COVID_project/COVID_project_data/poly_df.pkl"#
demo_file_path = "/home/jonno/polycentric-mobility/demo_data.pkl"
result_save_path = "/home/jonno/COVID_project/COVID_project_data/multi_gyration.pkl"

#!python /home/jonno/polycentric-mobility/main.py --data_path "{target_file_path}" --results_path "{result_save_path}" 


### Load test and demo data

In [10]:
test_data_df = pd.read_pickle("/home/jonno/COVID_project/COVID_project_data/poly_df.pkl")

In [5]:
demo_df = pd.read_pickle("/home/jonno/polycentric-mobility/demo_data.pkl")
print(demo_df)

      loc        lat        lon  time_spent  user                   geometry
0      53  55.830584  12.392003           1     1  POINT (12.39200 55.83058)
1       9  55.827128  12.389890           1     1  POINT (12.38989 55.82713)
2      23  55.728763  12.504619           1     1  POINT (12.50462 55.72876)
3      84  55.759575  12.362043           1     1  POINT (12.36204 55.75958)
4      88  55.773512  12.447521           1     1  POINT (12.44752 55.77351)
...   ...        ...        ...         ...   ...                        ...
1795  270  55.372064  12.000921           1     3  POINT (12.00092 55.37206)
1796  107  55.508729  11.492881           1     3  POINT (11.49288 55.50873)
1797  298  55.340643  11.977785           1     3  POINT (11.97779 55.34064)
1798   46  55.776968  12.327596           1     3  POINT (12.32760 55.77697)
1799   98  55.800256  12.394273           1     3  POINT (12.39427 55.80026)

[1800 rows x 6 columns]


### Comparing data types
The data types and the column names for the arguements are identical

In [11]:
test_data_df.dtypes

loc             int64
lat           float64
lon           float64
time_spent      int64
user            int64
dtype: object

In [7]:
demo_df.dtypes

loc              int64
lat            float64
lon            float64
time_spent       int64
user             int64
geometry      geometry
dtype: object

In [5]:
#Demo data succeeds
import time
start_time = time.time()
main(data_path = demo_file_path, results_path = result_save_path)
print("--- %s seconds ---" % (time.time() - start_time))

--- 103.04703950881958 seconds ---


In [5]:
#The real data fails
import time
start_time = time.time()
main(data_path = target_file_path, results_path = result_save_path,  max_k=6)
print("--- %s seconds ---" % (time.time() - start_time))

--- 23237.76428437233 seconds ---


In [7]:
pd.read_pickle(result_save_path).to_csv("/home/jonno/COVID_project/COVID_project_data/multi_gyration.csv")

In [12]:
multi_gyration = pd.read_pickle('/home/jonno/COVID_project/COVID_project_data/multi_gyration_test.pkl')
print(multi_gyration)

                                            com  \
user                                              
0     (51.50936611114374, -0.07897089589632829)   
1      (51.57028021391862, 0.10676118654879706)   
2       (51.5616122600484, 0.09884438706657804)   
3      (51.560455660524546, 0.1477798251896961)   
4      (51.55068770444214, 0.09347319403867203)   
5     (51.551550170741265, 0.11085328909507795)   
6      (51.54382176759679, 0.08350875388764721)   
7       (51.54708765584114, 0.0867602580508517)   
8      (51.54600246715115, 0.10311999488752895)   
9     (51.544359444615736, 0.10407477063155353)   

                                           tcom       rog  \
user                                                        
0     (51.51464830725228, -0.08204290624543803)  7.813708   
1      (51.57494813308364, 0.12754345935861824)  6.449312   
2      (51.56575968766957, 0.12343755320683983)  6.865981   
3      (51.56416987651819, 0.17221942235717372)  6.359004   
4      (51.5551017481